<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/Finding_the_EWT_of_buses_to_all_stop_points_in_a_given_linedID_and_direction_based_on_the_next_arrivals_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Finding the EWT of buses to a stop point in a given linedID and direction based on the next arrivals API

In [1]:
# Step 1: Get user inputs for line_id, stop_point_id, and direction
line_id = input('Enter the line id: ')
stop_point_id = input('Enter the stop point id: ')
direction = input('Enter the direction (inbound/outbound): ')
nbph = int(input('Enter the number of buses scheduled per hour: '))

# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions with error handling
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"

        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        # Fetch station name from the first item (assuming it's the same for all)
        station_name = data[0]['stationName']

        # Step 5: Parse the data
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + pd.Timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        # Step 6: Display the data in a readable form using pandas
        df = pd.DataFrame(predictions)

        # Sort by 'Expected Arrival (BST)'
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)

        # Convert 'Expected Arrival (HM)' to datetime format
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')

        # Calculate headway (considering only hour and minute)
        df['Headway (minutes)'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

        # Calculate AWT (Average Waiting Time) as half of headway
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)

        # Calculate WAWT as the product of headway and AWT/bus
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)

        # Format 'Expected Arrival (BST)' column to string for better readability, ignoring seconds and microseconds
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df, station_name

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None  # Return None if there's an error

# Function to create the dynamic timeline dashboard
def create_dashboard(df, station_name):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for the stop point at the right end
    fig.add_trace(go.Scatter(
        x=[current_time_bst],
        y=[0],
        mode='markers+text',
        marker=dict(size=40, symbol='square', color='black'),
        name='Stop Point',
        text=f"{current_time_bst}",  # Display only the current time below the square
        textposition='bottom center',
        hoverinfo='text'
    ))

    # Add annotations for the station name and stop point ID above the square
    fig.add_annotation(
        x=current_time_bst,
        y=0,
        text=f"{station_name}<br>{stop_point_id}",
        showarrow=False,
        font=dict(size=14, color="black"),
        yshift=40  # Adjust the yshift to move the label above the square
    )

    # Add circles for each bus moving towards the stop point
    y_gap = 0.01  # 1 cm gap (considering plotly's default sizing)
    y_positions = {}  # Dictionary to store y positions for each expected arrival time

    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']

        if expected_arrival in y_positions:
            y_pos = y_positions[expected_arrival] + y_gap
        else:
            y_pos = 0
            y_positions[expected_arrival] = y_pos

        y_positions[expected_arrival] = y_pos  # Update y position for current expected arrival

        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[y_pos],  # Y position adjusted to maintain a 1 cm gap
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=f"{row['Vehicle ID']}<br>{expected_arrival}",
            textposition='bottom center',
            name=row['Vehicle ID'],
            hoverinfo='text'
        ))

    # Define the x-axis range with the current time as origin and scaled such that 1 cell = 10 minutes
    x_start = datetime.now()
    x_end = x_start + timedelta(hours=1)  # Show 1 hour range

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(
            title='',
            showticklabels=True,
            tickformat='%H:%M',  # Format the ticks as HH:MM
            tickvals=[(x_start + timedelta(minutes=10 * i)).strftime('%H:%M') for i in range(7)],  # 6 cells = 60 minutes
            range=[x_start.strftime('%H:%M:%S'), x_end.strftime('%H:%M:%S')]
        ),
        yaxis=dict(showticklabels=False),  # Remove y-axis tick labels
        showlegend=True
    )

    return fig

# Main loop to fetch and display data
while True:
    df, station_name = fetch_arrival_predictions(line_id, stop_point_id, direction)

    if df is not None and station_name is not None:  # Check if data fetching was successful
        clear_output(wait=True)  # Clear previous output to update the table and dashboard

        # Calculate summary metrics
        total_wawt = df['WAWT'].sum()

        min_arrival = pd.to_datetime(df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
        max_arrival = pd.to_datetime(df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
        time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

        awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
        swt = round(60 / (nbph * 2), 2)
        ewt = round(awt - swt, 2)

        # Create summary DataFrame
        summary_df = pd.DataFrame({
            'Metric': ['Number of buses scheduled per hour (nbph)', 'Total WAWT (minutes)', 'Time difference between 1st and last observed buses (minutes)', 'AWT (minutes)', 'SWT (minutes)', 'EWT (minutes)'],
            'Value': [nbph, total_wawt, time_diff_minutes, awt, swt, ewt]
        })

        # Display the updated predictions
        print("Arrival Predictions:")
        display(df[['Line', 'Vehicle ID', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

        # Display the summary metrics
        print("\nSummary Metrics:")
        display(summary_df)

        # Create and display the dynamic timeline dashboard
        fig = create_dashboard(df, station_name)
        fig.show()

    else:
        # Display an error message if data fetching failed
        print("Failed to fetch data. Please check your inputs or try again later.")

    time.sleep(15)  # Wait for 15 seconds before the next update


Arrival Predictions:


<ipython-input-1-dc658988f992>:179: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-1-dc658988f992>:180: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Line,Vehicle ID,Stop Point,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
1,22,BV66VLE,490012375E,outbound,09:27:00,0.0,0.0,0.0
0,22,BV66VHR,490012375E,outbound,09:38:00,11.0,5.5,60.5



Summary Metrics:


,Metric,Value
0,Number of buses scheduled per hour (nbph),6.0
1,Total WAWT (minutes),60.5
2,Time difference between 1st and last observed ...,11.0
3,AWT (minutes),5.5
4,SWT (minutes),5.0
5,EWT (minutes),0.5


KeyboardInterrupt: 

In [2]:
# Step 1: Get user inputs for line_id and direction
line_id = input('Enter the line id: ')
direction = input('Enter the direction (inbound/outbound): ')
nbph = int(input('Enter the number of buses scheduled per hour: '))

# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions for all stop points on a line
def fetch_arrival_predictions(line_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"

        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        # Step 5: Parse the data
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + pd.Timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': item['naptanId'],
                'Station Name': item['stationName'],
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        # Step 6: Display the data in a readable form using pandas
        df = pd.DataFrame(predictions)

        # Sort by 'Expected Arrival (BST)'
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)

        # Convert 'Expected Arrival (HM)' to datetime format
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')

        # Calculate headway (considering only hour and minute)
        df['Headway (minutes)'] = df.groupby('Stop Point')['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

        # Calculate AWT (Average Waiting Time) as half of headway
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)

        # Calculate WAWT as the product of headway and AWT/bus
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)

        # Format 'Expected Arrival (BST)' column to string for better readability, ignoring seconds and microseconds
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None  # Return None if there's an error

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for each stop point
    stop_points = df['Stop Point'].unique()
    for stop_point in stop_points:
        stop_point_df = df[df['Stop Point'] == stop_point]
        station_name = stop_point_df['Station Name'].iloc[0]
        fig.add_trace(go.Scatter(
            x=[current_time_bst],
            y=[stop_point],
            mode='markers+text',
            marker=dict(size=40, symbol='square', color='black'),
            name=f'Stop Point {stop_point}',
            text=f"{current_time_bst}",  # Display only the current time below the square
            textposition='bottom center',
            hoverinfo='text'
        ))

        # Add annotations for the station name and stop point ID above the square
        fig.add_annotation(
            x=current_time_bst,
            y=stop_point,
            text=f"{station_name}<br>{stop_point}",
            showarrow=False,
            font=dict(size=14, color="black"),
            yshift=40  # Adjust the yshift to move the label above the square
        )

    # Add circles for each bus moving towards the stop point
    y_gap = 0.1  # Gap between buses
    y_positions = {}  # Dictionary to store y positions for each expected arrival time

    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        stop_point = row['Stop Point']

        if (expected_arrival, stop_point) in y_positions:
            y_pos = y_positions[(expected_arrival, stop_point)] + y_gap
        else:
            y_pos = stop_point
            y_positions[(expected_arrival, stop_point)] = y_pos

        y_positions[(expected_arrival, stop_point)] = y_pos  # Update y position for current expected arrival

        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[y_pos],  # Y position adjusted to maintain a gap
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=f"{row['Vehicle ID']}<br>{expected_arrival}",
            textposition='bottom center',
            name=row['Vehicle ID'],
            hoverinfo='text'
        ))

    # Define the x-axis range with the current time as origin and scaled such that 1 cell = 10 minutes
    x_start = datetime.now()
    x_end = x_start + timedelta(hours=1)  # Show 1 hour range

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(
            title='',
            showticklabels=True,
            tickformat='%H:%M',  # Format the ticks as HH:MM
            tickvals=[(x_start + timedelta(minutes=10 * i)).strftime('%H:%M') for i in range(7)],  # 6 cells = 60 minutes
            range=[x_start.strftime('%H:%M:%S'), x_end.strftime('%H:%M:%S')]
        ),
        yaxis=dict(
            title='Stop Points',
            tickvals=stop_points,
            ticktext=[df[df['Stop Point'] == sp]['Station Name'].iloc[0] for sp in stop_points]
        ),
        showlegend=True
    )

    return fig

# Main loop to fetch and display data
while True:
    df = fetch_arrival_predictions(line_id, direction)

    if df is not None:  # Check if data fetching was successful
        clear_output(wait=True)  # Clear previous output to update the table and dashboard

        # Calculate summary metrics
        summary_metrics = []

        for stop_point in df['Stop Point'].unique():
            stop_point_df = df[df['Stop Point'] == stop_point]
            total_wawt = stop_point_df['WAWT'].sum()

            min_arrival = pd.to_datetime(stop_point_df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
            max_arrival = pd.to_datetime(stop_point_df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
            time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

            awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
            swt = round(60 / (nbph * 2), 2)
            ewt = round(awt - swt, 2)

            summary_metrics.append({
                'Stop Point': stop_point,
                'Station Name': stop_point_df['Station Name'].iloc[0],
                'Total WAWT (minutes)': total_wawt,
                'Time difference between 1st and last observed buses (minutes)': time_diff_minutes,
                'AWT (minutes)': awt,
                'SWT (minutes)': swt,
                'EWT (minutes)': ewt
            })

        summary_df = pd.DataFrame(summary_metrics)

        # Display the updated predictions
        print("Arrival Predictions:")
        display(df[['Line', 'Vehicle ID', 'Stop Point', 'Station Name', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

        # Display the summary metrics
        print("\nSummary Metrics:")
        display(summary_df)

        # Create and display the dynamic timeline dashboard
        fig = create_dashboard(df)
        fig.show()

    else:
        # Display an error message if data fetching failed
        print("Failed to fetch data. Please check your inputs or try again later.")

    time.sleep(15)  # Wait for 15 seconds before the next update


<ipython-input-2-3bbbee3fb3d8>:189: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-2-3bbbee3fb3d8>:190: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-2-3bbbee3fb3d8>:189: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-2-3bbbee3fb3d8>:190: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-2-3bbbee3fb3d8>:189: UserWarning:

Could not infer format, so each element will be parsed individ

Arrival Predictions:


,Line,Vehicle ID,Stop Point,Station Name,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
128,22,LF67EWW,490006224KP,Duke of York Square,outbound,09:31:00,0.0,0.0,0.0
72,22,LJ62KFD,490005076W,Edith Grove / Worlds End,outbound,09:31:00,0.0,0.0,0.0
167,22,LJ62KFF,490011281S,Putney Common,outbound,09:32:00,0.0,0.0,0.0
116,22,YY66OZS,490014186W,Wandsworth Bridge Road,outbound,09:32:00,0.0,0.0,0.0
29,22,BV66VKB,490003923N,Berkeley Square,outbound,09:32:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
118,22,BV66VHZ,490000119K,Hyde Park Corner,outbound,10:01:00,10.0,5.0,50.0
170,22,LJ62KFF,490008435E,Putney Bridge Stn / Fulham High Street,outbound,10:01:00,11.0,5.5,60.5
134,22,BV66VLP,490005076W,Edith Grove / Worlds End,outbound,10:01:00,8.0,4.0,32.0
174,22,BV66VKB,490015119W,Berkeley Square,outbound,10:01:00,10.0,5.0,50.0



Summary Metrics:


,Stop Point,Station Name,Total WAWT (minutes),Time difference between 1st and last observed buses (minutes),AWT (minutes),SWT (minutes),EWT (minutes)
0,490006224KP,Duke of York Square,61.0,18.0,3.39,5.0,-1.61
1,490005076W,Edith Grove / Worlds End,178.0,30.0,5.93,5.0,0.93
2,490011281S,Putney Common,148.0,28.0,5.29,5.0,0.29
3,490014186W,Wandsworth Bridge Road,42.5,13.0,3.27,5.0,-1.73
4,490003923N,Berkeley Square,224.5,27.0,8.31,5.0,3.31
5,490003924F,Berkeley Street,110.5,21.0,5.26,5.0,0.26
6,490015145E,Beaufort Street,105.5,23.0,4.59,5.0,-0.41
7,490004809KD,Carlyle Square,169.0,24.0,7.04,5.0,2.04
8,490000206E,Cadogan Gardens / Sloane Square,66.0,18.0,3.67,5.0,-1.33
9,490007492E,Green Park / Constitution Hill,123.0,26.0,4.73,5.0,-0.27


KeyboardInterrupt: 

In [ ]:
# Step 1: Get user inputs for line_id and direction
line_id = input('Enter the line id: ')
direction = input('Enter the direction (inbound/outbound): ')
nbph = int(input('Enter the number of buses scheduled per hour: '))

# Step 2: Install required libraries (if not already installed)
!pip install requests pandas plotly dash

# Step 3: Import required libraries
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from IPython.display import display, clear_output
import plotly.express as px
import plotly.graph_objects as go
import pytz
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Function to convert time to BST
def convert_to_bst(dt):
    # Define UTC and BST timezones
    utc_tz = pytz.timezone('UTC')
    bst_tz = pytz.timezone('Europe/London')  # BST timezone for London

    # Convert input datetime to UTC timezone
    utc_dt = utc_tz.localize(dt)

    # Convert UTC time to BST
    bst_dt = utc_dt.astimezone(bst_tz)

    return bst_dt.strftime('%H:%M:%S')

# Function to fetch arrival predictions for all stop points on a line
def fetch_arrival_predictions(line_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals"

        params = {'direction': direction}

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors

        data = response.json()

        # Step 5: Parse the data
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + pd.Timedelta(hours=1)  # Convert GMT to BST
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': item['naptanId'],
                'Station Name': item['stationName'],
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')  # Format to HH:MM
            })

        # Step 6: Display the data in a readable form using pandas
        df = pd.DataFrame(predictions)

        # Sort by 'Expected Arrival (BST)'
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)

        # Convert 'Expected Arrival (HM)' to datetime format
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')

        # Calculate headway (considering only hour and minute)
        df['Headway (minutes)'] = df.groupby('Stop Point')['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60

        # Calculate AWT (Average Waiting Time) as half of headway
        df['AWT/bus (minutes)'] = (df['Headway (minutes)'] / 2).round(2)

        # Calculate WAWT as the product of headway and AWT/bus
        df['WAWT'] = (df['Headway (minutes)'] * df['AWT/bus (minutes)']).round(2)

        # Format 'Expected Arrival (BST)' column to string for better readability, ignoring seconds and microseconds
        df['Expected Arrival (BST)'] = df['Expected Arrival (BST)'].apply(lambda x: x.replace(second=0, microsecond=0).strftime('%H:%M:%S'))

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None  # Return None if there's an error

# Function to create the dynamic timeline dashboard
def create_dashboard(df):
    current_time_utc = datetime.utcnow()  # Get current UTC time
    current_time_bst = convert_to_bst(current_time_utc)  # Convert to BST

    # Create a timeline figure
    fig = go.Figure()

    # Add a big square for each stop point
    stop_points = df['Stop Point'].unique()
    for stop_point in stop_points:
        stop_point_df = df[df['Stop Point'] == stop_point]
        station_name = stop_point_df['Station Name'].iloc[0]
        fig.add_trace(go.Scatter(
            x=[current_time_bst],
            y=[stop_point],
            mode='markers+text',
            marker=dict(size=40, symbol='square', color='black'),
            name=f'Stop Point {stop_point}',
            text=f"{current_time_bst}",  # Display only the current time below the square
            textposition='bottom center',
            hoverinfo='text'
        ))

        # Add annotations for the station name and stop point ID above the square
        fig.add_annotation(
            x=current_time_bst,
            y=stop_point,
            text=f"{station_name}<br>{stop_point}",
            showarrow=False,
            font=dict(size=14, color="black"),
            yshift=40  # Adjust the yshift to move the label above the square
        )

    # Add circles for each bus moving towards the stop point
    y_gap = 0.1  # Gap between buses
    y_positions = {}  # Dictionary to store y positions for each expected arrival time

    for i, row in df.iterrows():
        expected_arrival = row['Expected Arrival (BST)']
        stop_point = row['Stop Point']

        if (expected_arrival, stop_point) in y_positions:
            y_pos = y_positions[(expected_arrival, stop_point)] + y_gap
        else:
            y_pos = stop_point
            y_positions[(expected_arrival, stop_point)] = y_pos

        y_positions[(expected_arrival, stop_point)] = y_pos  # Update y position for current expected arrival

        fig.add_trace(go.Scatter(
            x=[expected_arrival],
            y=[y_pos],  # Y position adjusted to maintain a gap
            mode='markers+text',
            marker=dict(size=12, color=px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]),
            text=f"{row['Vehicle ID']}<br>{expected_arrival}",
            textposition='bottom center',
            name=row['Vehicle ID'],
            hoverinfo='text'
        ))

    # Define the x-axis range with the current time as origin and scaled such that 1 cell = 10 minutes
    x_start = datetime.now()
    x_end = x_start + timedelta(hours=1)  # Show 1 hour range

    # Update layout
    fig.update_layout(
        title='Bus Positions by Expected Arrival Time',
        xaxis=dict(
            title='',
            showticklabels=True,
            tickformat='%H:%M',  # Format the ticks as HH:MM
            tickvals=[(x_start + timedelta(minutes=10 * i)).strftime('%H:%M') for i in range(7)],  # 6 cells = 60 minutes
            range=[x_start.strftime('%H:%M:%S'), x_end.strftime('%H:%M:%S')]
        ),
        yaxis=dict(
            title='Stop Points',
            tickvals=stop_points,
            ticktext=[df[df['Stop Point'] == sp]['Station Name'].iloc[0] for sp in stop_points]
        ),
        showlegend=True
    )

    return fig

# Main loop to fetch and display data
while True:
    df = fetch_arrival_predictions(line_id, direction)

    if df is not None:  # Check if data fetching was successful
        clear_output(wait=True)  # Clear previous output to update the table and dashboard

        # Calculate summary metrics
        summary_metrics = []

        for stop_point in df['Stop Point'].unique():
            stop_point_df = df[df['Stop Point'] == stop_point]
            total_wawt = stop_point_df['WAWT'].sum()

            min_arrival = pd.to_datetime(stop_point_df['Expected Arrival (BST)']).min().replace(second=0, microsecond=0)
            max_arrival = pd.to_datetime(stop_point_df['Expected Arrival (BST)']).max().replace(second=0, microsecond=0)
            time_diff_minutes = (max_arrival - min_arrival).total_seconds() / 60

            awt = round(total_wawt / time_diff_minutes, 2) if time_diff_minutes > 0 else 0
            swt = round(60 / (nbph * 2), 2)
            ewt = round(awt - swt, 2)

            summary_metrics.append({
                'Stop Point': stop_point,
                'Station Name': stop_point_df['Station Name'].iloc[0],
                'Total WAWT (minutes)': total_wawt,
                'Time difference between 1st and last observed buses (minutes)': time_diff_minutes,
                'AWT (minutes)': awt,
                'SWT (minutes)': swt,
                'EWT (minutes)': ewt
            })

        summary_df = pd.DataFrame(summary_metrics)

        # Display the updated predictions
        print("Arrival Predictions:")
        display(df[['Line', 'Vehicle ID', 'Stop Point', 'Station Name', 'Direction', 'Expected Arrival (BST)', 'Headway (minutes)', 'AWT/bus (minutes)', 'WAWT']])

        # Display the summary metrics
        print("\nSummary Metrics:")
        display(summary_df)

        # Create and display the dynamic timeline dashboard
        fig = create_dashboard(df)
        fig.show()

    else:
        # Display an error message if data fetching failed
        print("Failed to fetch data. Please check your inputs or try again later.")

    time.sleep(15)  # Wait for 15 seconds before the next update


Arrival Predictions:


<ipython-input-3-3bbbee3fb3d8>:189: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-3-3bbbee3fb3d8>:190: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-3-3bbbee3fb3d8>:189: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-3-3bbbee3fb3d8>:190: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

<ipython-input-3-3bbbee3fb3d8>:189: UserWarning:

Could not infer format, so each element will be parsed individ

,Line,Vehicle ID,Stop Point,Station Name,Direction,Expected Arrival (BST),Headway (minutes),AWT/bus (minutes),WAWT
81,22,BV66VJZ,490000119P,Hyde Park Corner Station,outbound,09:36:00,0.0,0.0,0.0
77,22,YY66OZH,490000173RF,Oxford Circus Stn / Margaret Street,outbound,09:36:00,0.0,0.0,0.0
10,22,BV66VHU,490015145W,Beaufort Street,outbound,09:36:00,0.0,0.0,0.0
53,22,YY66OZS,490008435W,Putney Bridge Stn / Fulham High Street,outbound,09:36:00,0.0,0.0,0.0
184,22,BV66VLH,490015575X,Parsons Green,outbound,09:36:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
97,22,LJ62KFF,490014186I,Wandsworth Bridge Road,outbound,10:04:00,8.0,4.0,32.0
176,22,YY66OYU,490000173RF,Oxford Circus Stn / Margaret Street,outbound,10:05:00,9.0,4.5,40.5
8,22,YY66OZH,490005073KC,Chelsea Old Town Hall,outbound,10:05:00,12.0,6.0,72.0
24,22,BV66VHR,490010534E,Old Church Street,outbound,10:05:00,9.0,4.5,40.5



Summary Metrics:


,Stop Point,Station Name,Total WAWT (minutes),Time difference between 1st and last observed buses (minutes),AWT (minutes),SWT (minutes),EWT (minutes)
0,490000119P,Hyde Park Corner Station,110.5,25.0,4.42,5.0,-0.58
1,490000173RF,Oxford Circus Stn / Margaret Street,140.5,29.0,4.84,5.0,-0.16
2,490015145W,Beaufort Street,137.0,22.0,6.23,5.0,1.23
3,490008435W,Putney Bridge Stn / Fulham High Street,68.0,16.0,4.25,5.0,-0.75
4,490015575X,Parsons Green,114.0,26.0,4.38,5.0,-0.62
5,490009370E,Worlds End Health Centre,94.5,23.0,4.11,5.0,-0.89
6,490011788W,Rumbold Road,146.0,22.0,6.64,5.0,1.64
7,490011278S,St Mary's Church / Putney Pier,122.5,21.0,5.83,5.0,0.83
8,490000173RG,Oxford Circus Station,100.0,20.0,5.00,5.0,0.00
9,490000093PA,Green Park Station,96.5,19.0,5.08,5.0,0.08
